In [1]:
from main import init_system
from ML_FKC_Detection import FKC_Predication
from pathselection import joinpathselection
from integrating_civilizer import *
from api.apiutils import Relation
api, reporting = init_system("/Users/emansour/elab/DAGroup/DataCivilizer/Aurum-GitHub/aurum-datadiscovery/models/mitdw/")

Loading: */Users/emansour/elab/DAGroup/DataCivilizer/Aurum-GitHub/aurum-datadiscovery/models/mitdw/*

### Help Menu

You can use the system through an **API** object. API objects are returnedby the *init_system* function, so you can get one by doing:

***your_api_object = init_system('path_to_stored_model')***

Once you have access to an API object there are a few concepts that are useful to use the API. **content** refers to actual values of a given field. For example, if you have a table with an attribute called __Name__ and values *Olu, Mike, Sam*, content refers to the actual values, e.g. Mike, Sam, Olu.

**schema** refers to the name of a given field. In the previous example, schema refers to the word__Name__ as that's how the field is called.

Finally, **entity** refers to the *semantic type* of the content. This is in experimental state. For the previous example it would return *'person'* as that's what those names refer to.

Certain functions require a *field* as input. In general a field is specified by the source name (e.g. table name) and the field name (e.g. attribute name). For example, if we are interested in finding content similar to the one of the attribute *year* in the table *Employee* we can provide the field in the following way:

field = ('Employee', 'year') # field = [<source_name>, <field_name>)
Took 1.1344671249389648 to load all data


# Story

We have 2400 tables in MIT data warehouse and among them 160 are semi-public. In this demo we will show how to find all the employees in MIT working in the degree granting department.

# 1-Discovering Relevant Tables

### I want to find tables about employees

In [8]:
res = api.table_name_search('Employee')
res.print_tables()

Employee_directory.csv


In [4]:
res.pretty_print_columns()

DB: MITDW                TABLE: Employee_directory.csv         FIELD: Full Name                     
DB: MITDW                TABLE: Employee_directory.csv         FIELD: Directory Full Name           
DB: MITDW                TABLE: Employee_directory.csv         FIELD: Primary Title                 
DB: MITDW                TABLE: Employee_directory.csv         FIELD: Department Name               
DB: MITDW                TABLE: Employee_directory.csv         FIELD: Office Location               
DB: MITDW                TABLE: Employee_directory.csv         FIELD: Krb Name                      
DB: MITDW                TABLE: Employee_directory.csv         FIELD: Email Address                 
DB: MITDW                TABLE: Employee_directory.csv         FIELD: Personal Url                  
DB: MITDW                TABLE: Employee_directory.csv         FIELD: Email Address Uppercase       
DB: MITDW                TABLE: Employee_directory.csv         FIELD: Department Number    

### I want to find tables that contain information about degree granting

In [5]:
res = api.schema_name_search("degree")
res.pretty_print_columns()

DB: MITDW                TABLE: Sis_department.csv             FIELD: Is Degree Granting            
DB: MITDW                TABLE: Hr_faculty_roster.csv          FIELD: Terminal Degree               
DB: MITDW                TABLE: Student_degree_program.csv     FIELD: Degree Code                   
DB: MITDW                TABLE: Student_degree_program.csv     FIELD: Degree Weight                 
DB: MITDW                TABLE: Student_degree_program.csv     FIELD: Degree Desc                   
DB: MITDW                TABLE: Student_degree_program.csv     FIELD: Degree Type                   
DB: MITDW                TABLE: Sis_course_description.csv     FIELD: Is Degree Granting            
DB: MITDW                TABLE: Student_degree_program.csv     FIELD: Degree Desc Short             
DB: MITDW                TABLE: Sis_course_description.csv     FIELD: Default Ultimate Degree       
DB: MITDW                TABLE: Student_degree_program.csv     FIELD: Degree Type Desc     

Not only I'm interested in tables with attributes that are related to **degree granting**, but also on those whose semantics are closer to **department**. I can capture that with a refined discovery query.

In [2]:
res1 = api.schema_name_search("degree")
res2 = api.table_name_search("department")
res1.set_table_mode()
res2.set_table_mode()
res3 = api.intersection(res1, res2)
res3.print_tables()

Sis_department.csv


In [3]:
table = "Sis_department.csv"
res = api.drs_from_table(table)
sim = api.similar_content_to(res)
sim.rank_certainty()
#sim.print_tables_with_scores()

Time to compute certainty scores: 0.005443096160888672
Time to compute coverage scores: 0.2095479965209961


**Sis_department.csv** seems a good candidate.

# 2-Finding Join Paths: Task-centric Recommandation for PKFK

I have found two tables that are relevant to my query: Who employees work in degree granting departments? To run my query, however, I'd like to have a **view** of both tables. Can I join them together?

I'm going to create a discovery query that finds join paths between two given tables. And I'm going to wrap up that function into another, for convenience.

In [4]:
def joinPaths(table1, table2):
    drs_t1 = api.drs_from_table(table1)
    drs_t2 = api.drs_from_table(table2)
    drs_t1.set_table_mode()
    drs_t2.set_table_mode()
    res = api.paths_between(drs_t1, drs_t2, Relation.PKFK, max_hops=2)
    #res = api.paths_between(drs_t1, drs_t2, Relation.CONTENT_SIM, max_hops=1)
    return res

## Task-centric Recommandation:  PKFK

In [6]:
t1 = "Drupal_employee_directory.csv"
t2 = "Employee_directory.csv"
resML = joinPaths(t1, t2)
FKC_Predication.classifyJoinPaths(resML)

Join Path                                                                                       Class      Propablity  
 "Employee_directory.csv@Last Name;Drupal_employee_directory.csv@Last Name"                     Recommended [ 0.  1.  0.]
 "Drupal_employee_directory.csv@Personal Url;Employee_directory.csv@Personal Url"               Recommended [ 0.  1.  0.]
 "Drupal_employee_directory.csv@Middle Name;Employee_directory.csv@Middle Name"                 Recommended [ 0.  1.  0.]
 "Drupal_employee_directory.csv@Office Location;Employee_directory.csv@Office Location"         Recommended [ 0.  1.  0.]
 "Drupal_employee_directory.csv@Hr Org Unit Title;Employee_directory.csv@Department Name"       Not Useful [ 0.  0.  1.]
 "Drupal_employee_directory.csv@First Name;Employee_directory.csv@Name Known By"                Not Useful [ 0.  0.  1.]
 "Drupal_employee_directory.csv@Mit Id;Employee_directory.csv@Mit Id"                           Real PKFK  [ 0.6  0.4  0. ]
 "Employee_directory.csv@F